# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [75]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
import json
warnings.filterwarnings('ignore')


# Import API key
from api_keys import geoapify_key

In [57]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,8.07,82,78,2.81,AU,1690114998
1,1,dudinka,69.4058,86.1778,20.43,61,35,6.14,RU,1690114998
2,2,bubaque,11.2833,-15.8333,26.99,83,96,6.49,GW,1690114999
3,3,keflavik,64.0049,-22.5624,13.24,71,75,2.57,IS,1690114999
4,4,avarua,-21.2078,-159.7750,24.03,100,75,5.66,CK,1690114999


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [101]:
%%capture --no-display

# Configure the map plot
vacation_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols=["Lng", "Lat", "City", "Humidity"],
    size="Humidity",
    color="City"
)


# Display the map
vacation_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [59]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 95 degrees but higher than 85.
# Wind speed less than 3 m/s.
# Cloudiness less than 10%.
ideal_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 35) & (city_data_df["Max Temp"] > 29) & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] < 50)].dropna()

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,georgetown,5.4112,100.3354,29.99,76,20,2.57,MY,1690114903
68,68,mermoz boabab,14.7065,-17.4758,32.31,66,20,1.03,SN,1690115008
112,112,martellago,45.5442,12.1607,30.58,49,20,2.57,IT,1690115015
138,138,taiyuan,37.8694,112.5603,29.92,35,0,1.00,CN,1690115018
166,166,gulu,2.7746,32.2990,30.29,35,18,1.47,UG,1690115021


### Step 3: Create a new DataFrame called `hotel_df`.

In [60]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
20,georgetown,MY,5.4112,100.3354,76,
68,mermoz boabab,SN,14.7065,-17.4758,66,
112,martellago,IT,45.5442,12.1607,49,
138,taiyuan,CN,37.8694,112.5603,35,
166,gulu,UG,2.7746,32.2990,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [99]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    radius = 10000
    # Set parameters to search for a hotel
    params = {
        "categories": "accommodation",
        "filter": f"circle:{lng},{lat},{radius}",
        "bias": f"proximity:{lng},{lat}",
        "apiKey": geoapify_key
        }

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
georgetown - nearest hotel: Page 63 hostel
mermoz boabab - nearest hotel: Le Shéhérazade
martellago - nearest hotel: B&B Mulino Orso Bianco
taiyuan - nearest hotel: 汾酒大厦
gulu - nearest hotel: No hotel found
suluktu - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
torit - nearest hotel: Safari Link
babolsar - nearest hotel: مشاور املاک مجلل
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
zaqatala - nearest hotel: Otel Zaqatala
arzgir - nearest hotel: No hotel found
itoman - nearest hotel: No hotel found
yuncheng - nearest hotel: 运城宾馆
changzhi - nearest hotel: No hotel found
cabugao - nearest hotel: No hotel found
nanyang - nearest hotel: 豫皖宾馆
urumqi - nearest hotel: Maitian International hostel
fuling - nearest hotel: Гостиница
sinop - nearest hotel: No hotel found
malanje - nearest hotel: Hotel PORTVGALIA
djibo - nearest hotel: No hotel found
zaysan - nearest hotel: Рахат қонақүй
xuchang - nearest hotel: first hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
20,georgetown,MY,5.4112,100.3354,76,Page 63 hostel
68,mermoz boabab,SN,14.7065,-17.4758,66,Le Shéhérazade
112,martellago,IT,45.5442,12.1607,49,B&B Mulino Orso Bianco
138,taiyuan,CN,37.8694,112.5603,35,汾酒大厦
166,gulu,UG,2.7746,32.2990,35,No hotel found
192,suluktu,KG,39.9337,69.5700,18,No hotel found
201,cabo san lucas,MX,22.8909,-109.9124,77,Comfort Rooms
212,torit,SS,4.4133,32.5678,37,Safari Link
304,babolsar,IR,36.7025,52.6576,66,مشاور املاک مجلل
325,zhangye,CN,38.9342,100.4517,12,Zhangye Hotel 张掖饭店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [102]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"],
    size="Humidity",
    color="City"
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)